# TOPS Stack Notebook

## Introduction
In this notebook, we will run all steps of processing Tops Stack. 
The detailed algorithm for stack processing of TOPS data can be find here:

+ Fattahi, H., P. Agram, and M. Simons (2016), A Network-Based Enhanced Spectral Diversity Approach for TOPS Time-Series Analysis, IEEE Transactions on Geoscience and Remote Sensing, 55(2), 777-786, doi:[10.1109/TGRS.2016.2614925](https://ieeexplore.ieee.org/abstract/document/7637021).

-----------------------------------

#### Note
Be sure to have ~/.netrc file include valid credentials for urs.earthdata.nasa.gov.

This notebook can process TOPS stacks for given spatial and temporal constraints.

**Input Parameters**

- **lat_min, lat_max, lon_min, lon_max**: min/max latitude and longitude of Region of Interest.


In [6]:
lat_min = 35.57793709766442
lat_max = 36.21619047354823
lon_min = -84.70058767311058
lon_max = -83.94773267597341

print(lat_min, lat_max, lon_min, lon_max)

35.57793709766442 36.21619047354823 -84.70058767311058 -83.94773267597341


## Initialization and set up

You might have to modify 00-init.sh

In [8]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh


Coords:
34.285041 37.501509 -86.721161 -83.227722 34 38 -87 -83



topsStack requires certain input to run which are SLCs, Orbits, Dems etc.

## Fetch DEM

In [ ]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/03-get-dems.sh

## Create run scripts and fetch orbits

The scripts provides support for Sentinel-1 TOPS stack processing. Currently supported workflows include a coregistered stack of SLC, interferograms, offsets, and coherence. 

`stackSentinel.py` generates all configuration and run files required to be executed on a stack of Sentinel-1 TOPS data. When stackSentinel.py is executed for a given workflow (-W option) a **configs** and **run_files** folder is generated. No processing is performed at this stage. Within the run_files folder different run\_#\_description files are contained which are to be executed as shell scripts in the run number order. Each of these run scripts call specific configure files contained in the “configs” folder which call ISCE in a modular fashion. The configure and run files will change depending on the selected workflow. To make run_# files executable, change the file permission accordingly (e.g., `chmod +x run_01_unpack_slc`).

```bash
stackSentinel.py -H     #To see workflow examples,
stackSentinel.py -h     #To get an overview of all the configurable parameters
```

Required parameters of stackSentinel.py include:

```cfg
-s SLC_DIRNAME          #A folder with downloaded Sentinel-1 SLC’s. 
-o ORBIT_DIRNAME        #A folder containing the Sentinel-1 orbits. Missing orbit files will be downloaded automatically
-a AUX_DIRNAME          #A folder containing the Sentinel-1 Auxiliary files
-d DEM_FILENAME         #A DEM (Digital Elevation Model) referenced to wgs84
```

In the following, different workflow examples are provided. Note that stackSentinel.py only generates the run and configure files. To perform the actual processing, the user will need to execute each run file in their numbered order.

In all workflows, coregistration (-C option) can be done using only geometry (set option = geometry) or with geometry plus refined azimuth offsets through NESD (set option = NESD) approach, the latter being the default. For the NESD coregistrstion the user can control the ESD coherence threshold (-e option) and the number of overlap interferograms (-O) to be used in NESD estimation.


In [ ]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/04-make-run-scripts.sh
sh ${toolDir}/05-make-run-scripts.sh


## Unpack TOPO reference

Unpacks the reference SLC files using ISCE readers, also unpackaging the antenna elevation pattern correction file if necessary. This run file also produces the reference geometry files that are consumed downstream.

Includes commands to unpack Sentinel-1 TOPS SLCs using ISCE readers. For older SLCs which need antenna elevation pattern correction, the file is extracted and written to disk. For newer version of SLCs which don’t need the elevation antenna pattern correction, only a gdal virtual “vrt” file (and isce xml file) is generated. The “.vrt” file points to the Sentinel SLC file and reads them whenever required during the processing. If a user wants to write the “.vrt” SLC file to disk, it can be done easily using gdal_translate (e.g. gdal_translate –of ENVI File.vrt File.slc). 
The “run_01_unpack_slc_topo_reference” also includes a command that refers to the config file of the stack reference, which includes configuration for running topo for the stack reference. Note that in the pair-wise processing strategy one should run topo (mapping from range-Doppler to geo coordinate) for all pairs. However, with stackSentinel, topo needs to be run only one time for the reference in the stack. 


In [ ]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-01.sh

## Unpack secondary SLC

Unpack Secondary SLCs
In a manner similar to the SLCs in the step above, this step unpacks the secondary SLCs from each of the input SLC zip files.

In [ ]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-02-0.sh

## SLC noise calibration

This step runs radiometric and thermal noise calibration using vh-pol data against the SLCs.

In [ ]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-02-5.sh

## Discard subswaths not to be used

In [ ]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-02-9.sh

## Average baseline

Computes average baseline for the stack. These baselines are not used for processing anywhere. They are only an approximation and can be used for plotting purposes.

In [ ]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-03.sh

## fullBurst geo2rdr

This step estimates geometrical offsets between secondary burst overlaps and the stack reference burst overlaps. The secondaries are then resampled to the stack reference burst overlaps.

In [ ]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-04.sh

## fullBurst resample

Using orbit and DEM data, this run file computes geometrical offsets among all secondary SLCs and the stack reference. These offsets, with the misregistration time series are used for precise coregistration of each burst SLC.


In [ ]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-05.sh

## Extract stack valid region

In [ ]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-06.sh

## Merge reference secondary SLC

This step merges all bursts for the reference and coregistered SLCs. Geometry files are also merged. Using orbit and DEM, geometrical offsets among all secondary SLCs and the stack reference is computed. The geometrical offsets, together with the misregistration time-series (from previous step) are used for precise coregistration of each burst SLC. 

In [ ]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-07.sh

## Create dataset

The final step ‘packages’ up the results into a single sub-directory named as coregistered_slcs-<start_date>-<end_date> and generates json files describing the dataset (e.g. the polygon  coordinates of the bounding box) and associated metadata (e.g. included SLCs).

More information about metadata files can be found at : https://hysds-core.atlassian.net/wiki/spaces/HYS/pages/389349377/Product+Metadata+Files

In [ ]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

# fake env var PROCESSING_START for now
export PROCESSING_START=$(date +%FT%T)

sh ${toolDir}/07-make-datasets.sh